# Import modules

In [ ]:
import stlab
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy
from scipy.optimize import *
import time
import sys 
import io
import os
from IPython.display import display, Javascript
from shutil import *
from stlab.devices.RS_SGS100A import RS_SGS100A
from stlab.devices.Keysight_N5183B import Keysight_N5183B

from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
from Configuration_BMDevice_FreqMixer import config, RR_1_IF,RO_lo, readout_len, R1_RS

# Configure QM unit

In [ ]:
qmm = QuantumMachinesManager()
qm = qmm.open_qm(config)

# Define microwave sources

In [ ]:
RR = RS_SGS100A("TCPIP::169.254.184.193::INSTR", reset=True,verb=True) 
RR.EXTref()
RR.RFon()
RR.setCWpower(23)
RR.setCWfrequency(RO_lo)
# RR.write(':SOURce:IQ:IMPairment:LEAKage:I ' + R1_RS[0])
# RR.write('SOURce:IQ:IMPairment:LEAKage:Q ' + R1_RS[1])
# RR.write(':SOURce:IQ:IMPairment:IQRatio:MAGNitude ' + R1_RS[2])
# RR.write(':SOURce:IQ:IMPairment:QUADrature:ANGLe ' + R1_RS[3])
# RR.IQon()
# RR.write(':SOURce:IQ:IMPairment:STATe ON')

#LO for downconversion
MXG = Keysight_N5183B(addr='TCPIP::192.168.1.91::INSTR',reset=True,verb=True)
MXG.RFon()
MXG.setCWpower(17)
MXG.setCWfrequency(RO_lo)
MXG.INTref()

# QUA Program

In [ ]:
# n_max = 10000
n_max = 5000
f_min=RR_1_IF-20e6  
f_max=RR_1_IF+20e6 #+25.0e6 
#df=0.05e6
df = 0.3e6
f_vec = np.arange(f_min, f_max+df/2, df)

a_min=0.1
a_max= 1
da = 0.030
# a_min=0.2
# a_max=0.25
# da=0.025
a_vec=np.arange(a_min,a_max+da/2,da)

print(len(f_vec))
print(len(a_vec))

In [ ]:
with program() as res_spec:
    
    n = declare(int)
    f = declare(int)
    a = declare(fixed)
    
    I=declare(fixed)
    Q=declare(fixed)
    
    I_st = declare_stream()
    Q_st = declare_stream()
        
    with for_(n, 0, n<n_max, n+1):     # averaging
        
        with for_(f, f_min, f<f_max+df/2, f+df):
            
            with for_(a,a_min,a<a_max+da/2,a+da):  # power
                update_frequency("RR_1", f)
                wait(int(5*readout_len/4), "RR_1")
                measure('readout'*amp(a), 'RR_1', None, demod.full("integW_cos", I, "out1"),
                                                        demod.full("integW_sin", Q,"out1"))
            
                save(I, I_st)
                save(Q, Q_st)
            
    with stream_processing():
        I_st.buffer(len(f_vec), len(a_vec)).average().save("I")
        Q_st.buffer(len(f_vec), len(a_vec)).average().save("Q")

In [ ]:
job = qm.execute(res_spec, duration_limit=0, data_limit=0) 

In [ ]:
res_handles= job.result_handles
res_handles.wait_for_all_values()

# Fetch data and save to .dat

In [ ]:
I_handle = res_handles.get("I")
Q_handle = res_handles.get("Q")

I = I_handle.fetch_all()
Q = Q_handle.fetch_all()

In [ ]:
prefix = 'S' #prefix for measurement folder name.  Can be anything or empty
idstring = f'R1_Pow_Sweep'

data_Re=I_handle.fetch_all()
data_Im=Q_handle.fetch_all()
data_Sig=np.abs(data_Re + 1j*data_Im)
data_Amp=20*(np.log10(np.abs(data_Sig)))
data_Ph=np.unwrap(np.arctan(data_Im/data_Re))
data=np.asarray([(RO_lo+(f_vec.reshape(len(f_vec),1)+a_vec)-a_vec).T,np.round(((a_vec.reshape(len(a_vec),1))+f_vec)-f_vec,3),data_Re.T,data_Im.T,data_Sig.T,data_Amp.T,data_Ph.T])

for i,amps in enumerate(a_vec):
    print(i,amps)
    data_dict={'Frequency (Hz)':data[0][i],
           'Resonator Pulse Amplitude':data[1][i],
           'Real':data[2][i],
           'Imaginary':data[3][i],
           'Signal':data[4][i],
           'Amplitude (dB)':data[5][i],
           'Phase (rad)':data[6][i]
    }
    if i==0:    
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        myfile=stlab.newfile(prefix,idstring,data_dict.keys(),autoindex=True, git_id = False)
        
        output = new_stdout.getvalue()
        sys.stdout = old_stdout
        print(output)
        M_ind = output.find("Measurement Name")
        M_name = output[M_ind+len('Measurement Name:  '):-1]
    stlab.savedict(myfile,data_dict)
    stlab.metagen.fromarrays(myfile,data_dict['Frequency (Hz)'],a_vec[0:i+1],xtitle='Frequency (Hz)',ytitle='Resonator Pulse Amplitude',colnames=list(data_dict))

# Processing/Plotting

In [ ]:
#Resonator LO power = -15 dBm

sig = I + 1j*Q

power = 20*(np.log10(np.abs(sig)))#+10*np.log10(1000/50)#np.abs(sig)   #equation from wiki
power2=power/np.mean(power,axis=0)                                    #norm lbl
power3=scipy.ndimage.gaussian_filter((power/np.mean(power,axis=0)),[1,3]) #gaussian filtered norm lbl
power4=power-np.mean(power,axis=0)                                    #sub lbl 
power5=scipy.ndimage.gaussian_filter(power,[1,3])-np.mean(power,axis=0)#gaussian filtered sub lbl
power6=scipy.ndimage.gaussian_filter(power,[1,3]) #just gaussian filtered
phase = np.unwrap(np.arctan(Q/I))

plt.figure(num=None, figsize=(8, 6), dpi=100)
X, Y = np.meshgrid((RO_lo+f_vec)/1e9, a_vec)
plt.contourf(X,Y, power.T,levels=100,norm=colors.PowerNorm(gamma=1.5),cmap='RdBu')
plt.xlabel('Frequency (GHz)')
plt.title('Resonator Power Spectroscopy, t_RO={}ns'.format(readout_len))
#plt.xticks(X[0,0],X[0,-1])
plt.ylabel('Pulse Amplitude')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Normalized Signal Amplitude', rotation=270)

In [ ]:
print(a_vec[4])
#plt.plot(RO_lo+f_vec,power[:,0])

plt.figure(figsize = (8,6))
plt.plot(RO_lo+f_vec,power[:,20])

# Save this file and configuration file to measurement folder

In [ ]:
#save notebook
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
#define document paths
meas_path = os.path.join(os.getcwd(),M_name)

current_nb_path = os.path.join(os.getcwd(),nb_name)
save_nb_path = os.path.join(meas_path,nb_name)

current_config_path = os.path.join(os.getcwd(), 'Configuration_BMDevice_FreqMixer.py')
save_config_path = os.path.join(meas_path, 'Configuration_BMDevice_FreqMixer.py')

#copy to measurement folder 
copy2(current_nb_path,save_nb_path);
copy2(current_config_path,save_config_path);